<a href="https://colab.research.google.com/github/yelenana/machine_learning/blob/main/Rybchynska_Olena__%22HW_2_2_%D0%9B%D0%BE%D0%B3%D1%96%D1%81%D1%82%D0%B8%D1%87%D0%BD%D0%B0_%D1%80%D0%B5%D0%B3%D1%80%D0%B5%D1%81%D1%96%D1%8F_%D0%B7_scikit_learn_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

В цьому домашньому завданні ми побудуємо рішення задачі логістичної регресії і створимо ваш перший submission на змагання на Kaggle. Усі кроки були розглянуті в лекції та містяться в ноутбуці `Логістична регресія з ScikitLearn. Повна ML задача.ipynb`, але тепер ми їх адаптуємо до іншої задачі - все як в реальному житті, коли ви знайшли код у відкритих джерелах і адаптуєте на проєкті 😉

**Погнали! 🚀🚀🚀**

-----------



0. Завантажте дані `train.csv`, `test.csv`, `sample_submission.csv` зі змагання на Kaggle - шукайте посилання в уроці [Запрошення до участі у Kaggle-змаганні.](https://data-loves.kwiga.com/courses/machine-learning-dlia-liudei/domashnie-zavdannia-zmagannia-z-kaggle) Для завантаження потрібно долучитись до змагання (натиснути кнопку "Join").

1. Зчитайте дані `train.csv` в pandas dataframe та розбийте їх на train і validation піднабори.

  * Є такий параметр в методі train_test_split як stratified. Можливо він буде Вам корисним при розбитті даних в цій задачі. Пропоную Вам почитати про нього в документації і спробувати скористатись.



---



In [ ]:
!pip install opendatasets --upgrade --quiet

In [ ]:
import opendatasets as od

In [ ]:
od.version()

In [ ]:
dataset_url = 'https://www.kaggle.com/datasets/barelydedicated/bank-customer-churn-modeling/sample_data'

In [ ]:
od.download(dataset_url)


In [ ]:
import os

In [ ]:
!ls

In [ ]:
data_dir = './bank-customer-churn-modeling'

In [ ]:
os.listdir(data_dir)

In [ ]:
train_csv = data_dir + '/Churn_Modelling.csv'

Завантажимо дані за допомогою Pandas

In [ ]:
import pandas as pd

In [ ]:
raw_df=pd.read_csv(train_csv)

In [ ]:
raw_df

In [ ]:
raw_df.info()

Бачимо, що всі наші дані заповнені, нама строчок зі значенням NA



---



2. Визначіть назви колонок, які сформують вхідні незалежні дані, в змінній `input_cols` та назву колонки , яка є цільовою, в змінній `target_col`. Використовуючи `input_cols` та `target_col` створіть набори даних `train_inputs`, `train_targets`, `val_inputs`, `val_targets` - де `inputs` - вхідні дані (датафрейм), `targets` - дані цільової колонки відповідного піднабору.



---



In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_val_df, test_df = train_test_split(raw_df, test_size=0.2, random_state=42)
train_df, val_df = train_test_split(train_val_df, test_size=0.25, random_state=42)

In [ ]:
print ('train_df',train_df.shape)
print ('val_df',val_df.shape)
print ('test_df',test_df.shape)

Створюємо список вхідних  стовпців і цільової змінної

In [ ]:
input_cols = list(train_df.columns)[3:-1]
target_col = 'Exited'

Перші 3 колонки не включаємо, бо це не має смислу

In [ ]:
print(input_cols)
print(target_col)

Створюємо набори вхідних даних і цільових значень для навчального, тестового та валідаційного набору

In [ ]:
train_inputs = train_df[input_cols].copy()
train_targets = train_df[target_col].copy()

In [ ]:
val_inputs = val_df[input_cols].copy()
val_targets = val_df[target_col].copy()

In [ ]:
test_inputs = test_df[input_cols].copy()
test_targets = test_df[target_col].copy()



---



3. Визначіть з допомогою `pandas.DataFrame.select_dtypes` назви колонок, які є числовими і категоріальними. Запишіть результати в `numeric_cols` та `categorical_cols` відповідно.



---



In [ ]:
import numpy as np

In [ ]:
train_inputs.head()

In [ ]:
numeric_cols = train_inputs.select_dtypes(include=np.number).columns.tolist()
categorical_cols = train_inputs.select_dtypes('object').columns.tolist()

In [ ]:
numeric_cols, categorical_cols

In [ ]:
train_inputs[numeric_cols].describe( ).round(2)

In [ ]:
train_inputs[categorical_cols].nunique()

4. Зробіть підготовку даних в категоріальних і числових колонках для входу в модель виходячи з освоєного на даний момент на курсі. Варіанти пеердобробки, з якими ми вже знайомі:
  - масштабування числових ознак,
  - заповнення пропущених значень, якщо вони є,
  - кодування категоріальних ознак, вилучення ознак, якщо ми вважаємо, що ці дані нерелевантні для моделі аби ми не бачимо способу як їх так закодувати, аби подати на вхід моделі.

  Тут є багато варіантів, як Ви побудуєте передобробку, важливо аби
  1) передобробка мала сенс (немає сенсу заповнювати дані константою, яку ми не зрозуміло звідки взяли)   
  2) має бути виконана принаймні одна дія передобробки
  3) обробка маэ бути виконана коректно і для тренувальних, і для вал. даних
  4) чим вищий у Вас врешті вийде ROС AUC скор, тим більше шансів перемогти у змаганні :)

  Опишіть рішення проведення кожного з кроків передобробки. Для чого ви його робите? По завершенню виведіть по 5 значен з `train_inputs`, `val_inputs` використовуючи `display()`.



---
1. Самим першим етапом підготовки даних було вилучення з розгляду стовпців  RowNumber,	CustomerId,	Surname,бо вони не несуть для подальшого аналізу ніякого смислового навантаження
2. Наступний необхідний крок: оцифрувати категріальні ознаки, а це в нас Geography та Gender


In [ ]:
gender_codes = {'Female': 0, 'Male': 1}
train_inputs['Gender'] = train_inputs['Gender'].map(gender_codes)
val_inputs['Gender'] = val_inputs['Gender'].map(gender_codes)
test_inputs['Gender'] = test_inputs['Gender'].map(gender_codes)

Так як гендер перевели у числовий вигляд, то перевизначимо числові і категоріальні ознаки

In [ ]:
numeric_cols = train_inputs.select_dtypes(include=np.number).columns.tolist()
categorical_cols = train_inputs.select_dtypes('object').columns.tolist()

In [ ]:
numeric_cols

In [ ]:
categorical_cols

In [ ]:
train_inputs.head(4)

Бачимо що кореляція між гендером та нашим таргетом незначна, але все ж таки цікаво буде проаналізувати, як це буде впливати на загальну модель.

In [ ]:
train_inputs.Gender.corr(train_targets)

Переведемо також у числові значення стовпчика Geography

In [ ]:
from sklearn import preprocessing
enc = preprocessing.OneHotEncoder(sparse_output=False)
enc.fit(train_inputs[['Geography']])
enc.categories_


In [ ]:
one_hot = enc.transform(train_inputs[['Geography']])
train_inputs[enc.categories_[0]] = one_hot
train_inputs

In [ ]:
one_hot = enc.transform(val_inputs[['Geography']])
val_inputs[enc.categories_[0]] = one_hot
val_inputs.head()

In [ ]:
one_hot = enc.transform(test_inputs[['Geography']])
test_inputs[enc.categories_[0]] = one_hot
test_inputs.head()

Промасштабуємо дані

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(train_inputs[numeric_cols])
train_inputs[numeric_cols] = scaler.transform(train_inputs[numeric_cols])
val_inputs[numeric_cols] = scaler.transform(val_inputs[numeric_cols])
test_inputs[numeric_cols] = scaler.transform(test_inputs[numeric_cols])

In [ ]:
train_inputs[numeric_cols].describe().round(2)



---



5. Збережіть дані, що містяться в змінних `train_inputs`, `train_targets`, `val_inputs`, `val_targets` на диск в форматі `parquet`. В кінці виведіть список файлів в папці, в яку Ви зберегли файли і перевірте, чи файли збереглись. Вивести список файлів можна з `os.listdir(your_dir)` або `!ls your_dir`



---



In [ ]:
!pip install pandas pyarrow --quiet

In [ ]:
output_dir = 'logistic_regression_data'
os.makedirs(output_dir, exist_ok=True)

In [ ]:
train_inputs.to_parquet(os.path.join(output_dir, 'train_inputs.parquet'))
val_inputs.to_parquet(os.path.join(output_dir, 'val_inputs.parquet'))
test_inputs.to_parquet(os.path.join(output_dir, 'test_inputs.parquet'))

In [ ]:
%%time
pd.DataFrame(train_targets).to_parquet(os.path.join(output_dir, 'train_targets.parquet'))
pd.DataFrame(val_targets).to_parquet(os.path.join(output_dir, 'val_targets.parquet'))
pd.DataFrame(test_targets).to_parquet(os.path.join(output_dir, 'test_targets.parquet'))

In [ ]:
print(f"\nВміст папки '{output_dir}':")
print(os.listdir(output_dir))

In [ ]:
train_inputs = pd.read_parquet(os.path.join(output_dir, 'train_inputs.parquet'))
val_inputs = pd.read_parquet(os.path.join(output_dir, 'val_inputs.parquet'))
test_inputs = pd.read_parquet(os.path.join(output_dir, 'test_inputs.parquet'))

train_targets = pd.read_parquet(os.path.join(output_dir, 'train_targets.parquet'))
val_targets = pd.read_parquet(os.path.join(output_dir, 'val_targets.parquet'))
test_targets = pd.read_parquet(os.path.join(output_dir, 'test_targets.parquet'))

Подивимось на дані, чи дійсно з ними все в порядку

In [ ]:
print('train_inputs',train_inputs.shape)
print('train_targets',train_targets.shape)
print('val_inputs',val_inputs.shape)
print('val_targets',val_targets.shape)
print('test_inputs',test_inputs.shape)
print('test_targets',test_targets.shape)

In [ ]:
train_inputs.head()

In [ ]:
train_targets.head()



---



6. З підготовлених вхіжних даних створіть `X_train`, `X_val` набори, які містять лише колонки, які підуть на вхід моделі. Навчіть модель бінарної класифікації з допомогою `LogisticRegression` в `scikit-learn`.



---



In [ ]:
X_train = train_inputs.drop(columns=['Geography'])

X_train.head()

In [ ]:
X_val= val_inputs.drop(columns=['Geography'])
X_val.head()

In [ ]:
X_test= test_inputs.drop(columns=['Geography'])
X_test.head()

Натренуємо модель

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
model=LogisticRegression(solver='liblinear')

In [ ]:
model.fit(X_train, train_targets)

In [ ]:
model.coef_.round(2)

In [ ]:
train_preds = model.predict(X_train)
train_preds



---



7. Виведіть Confusion matrix, ROC Curve, метрику AUROC та метрику F1 Score для порога класифікації 0.5 для побудованої моделі на тренувальних та валідаційних даних. Що можете сказати про якіст моделі? Вона хороша, погана, задовільна?

---

Confusion matrix:

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(train_targets, train_preds,normalize='true')

 F1 Score:

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
preds = model.predict(X_train)
f1_score(train_targets, preds)

In [ ]:
from sklearn.metrics import roc_curve,auc
import matplotlib.pyplot as plt

def compute_auroc_and_build_roc(inputs, targets, name=''):
    # Predict probabilities
    y_pred_proba = model.predict_proba(inputs)[:, 1]

    # Compute ROC curve
    fpr, tpr, thresholds = roc_curve(targets, y_pred_proba, pos_label=1)

    # Compute AUROC
    roc_auc = auc(fpr, tpr)
    print(f'AUROC for {name}: {roc_auc:.2f}')

    # Plot the ROC curve
    plt.figure()
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'Receiver Operating Characteristic (ROC) Curve for {name}')
    plt.legend(loc="lower right")
    plt.show()

# Виклики функції
compute_auroc_and_build_roc(X_train, train_targets, 'Training')
compute_auroc_and_build_roc(X_val, val_targets, 'Validation')

Висновки: Дуже висока специфічність (TN_rate): 96.92% правильно передбачених негативних випадків. Модель чудово ідентифікує, коли подія не відбудеться.

Дуже низька чутливість / повнота (TP_rate / Recall): 19.98% правильно передбачених позитивних випадків. Це означає, що 80.02% фактичних позитивних випадків були хибно класифіковані як негативні (False Negatives). Модель майже повністю пропускає позитивні події.
Модель надзвичайно упереджена до прогнозування негативного класу. Це є результатом сильного дисбалансу класів, де негативний клас є значно поширенішим(див.нижче). Модель "навчилася" мінімізувати помилки, прогнозуючи найчастіший клас.А найчастіший клас у нас це Exited=0 (79%)

In [ ]:
train_targets.value_counts(normalize=True)

In [ ]:
import pandas as pd
pd.Series(train_preds).value_counts(normalize=True)

Низьке значення F1-score (0.30) підтверджує, що модель не є ефективною для передбачення позитивного класу. Вона страждає від великої кількості хибно-негативних спрацьовувань (дуже низька повнота), що сильно впливає на F1-score.

F1-score підтверджує проблему, виявлену Confusion matrix: модель нездатна адекватно виявляти позитивні випадки.

AUROC:0.76 для тренувального набору та 0.78 для валідаційного набору є помірними показниками. Те, що AUROC для валідації (0.78) трохи вищий за тренувальний (0.76), може вказувати на відсутність сильного перенавчання (overfitting) і що модель узагальнює досить добре на нових даних.
Загалом: Модель має потенціал для розрізнення класів (AUROC ~0.77 є "нормальним", не ідеальним, але й не випадковим). Однак, модель надто упереджена до прогнозування негативного класу, що робить її неефективною для виявлення позитивних випадків.




---



8. Створіть передбачення моделі, яка передбачає завжди мажоритарний клас (той, якого більше в тренувальних даних). Порахуйте Accuracy на тренувальних та валідаційних даних для цієї моделі і тої, що ми натренували в п.6. Зробіть заключення про якість нашої моделі: якість хороша, погана, задовільна?



---



In [ ]:
#Створимо передбачення для "Majority Class Classifier"


In [ ]:
majority_class = train_targets.value_counts().idxmax()
print(f"Мажоритарний клас у тренувальних даних: {majority_class}")

Як вище і було вже розраховано, що Exited=0 (79% даних)

In [ ]:
# Для тренувальних даних
majority_train_preds = np.full(len(train_targets), majority_class)

# Для валідаційних даних
majority_val_preds = np.full(len(val_targets), majority_class)

In [ ]:
majority_train_preds

In [ ]:
majority_val_preds

In [ ]:
# Рахуємо Accuracy для "наївної" моделі (Majority Class Classifier) ---
from sklearn.metrics import accuracy_score
accuracy_majority_train = accuracy_score(train_targets, majority_train_preds)
accuracy_majority_val = accuracy_score(val_targets, majority_val_preds)

In [ ]:
accuracy_majority_train

In [ ]:
accuracy_majority_val

In [ ]:
# порівняємо зі значенням для навченої моделі
accuracy_score(train_targets, train_preds)

Висновок; модель, отримана після навчання (0.8113) дещо краща за модель мажоритарного класу(0.7948). Різниця становить близько 1.65%. Той факт, що Majority Class Classifier показує точність майже 80%, знову ж таки підтверджує, що у даних існує сильний дисбаланс класів - мажоритарний становить близько 80% всіх зразків



---



9. Збережіть модель і інші обʼєкти, які ви хотіли б зберегти, з використанням бібліотеки joblib в файл `log_reg.joblib`. Потім завантажте модель з файлу в змінну `model_2` (цим ми тренуємось завантажувати модель з файлу на майбутнє).



---



In [ ]:
import joblib


In [ ]:
churn_prediction_data = {
    'model': model,
    'scaler': scaler,
    'encoder':enc,
    'numeric_cols': numeric_cols,
    'categorical_cols': categorical_cols,
}

In [ ]:
joblib.dump(churn_prediction_data, 'log_reg.joblib')

In [ ]:
data = joblib.load('log_reg.joblib')
model_2 = data['model']
model_2

10. Ознайомтесь з роботою наведеної функції `predict_raw_df`. Ця функція робить підготовку даних (масштабування числових колонок і кодування категоріальних) і виводить ймовірності належності до класу 1 для кожного рядка в наданому наборі.

Доповніть або внесіть зміни у функцію з врахуванням вашого препроцесингу даних.

In [ ]:
numeric_cols, categorical_cols

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd

def predict_raw_df(scaler, encoder, numeric_cols, categorical_cols, model, input_df: pd.DataFrame):

    processed_df = input_df.copy()

    # Apply gender mapping
    gender_codes = {'Female': 0, 'Male': 1}
    processed_df['Gender'] = processed_df['Gender'].map(gender_codes)

    # Separate numeric and categorical columns
    numeric_data = processed_df[numeric_cols]
    categorical_data = processed_df[categorical_cols]

    # Scale numeric columns
    scaled_numeric_data = scaler.transform(numeric_data)
    scaled_numeric_df = pd.DataFrame(scaled_numeric_data, index=processed_df.index, columns=numeric_cols)


    # Encode geography column
    encoded_geo_cols_with_prefix = encoder.get_feature_names_out(['Geography'])
    one_hot_geo = encoder.transform(categorical_data[['Geography']])

    rename_dict = {col: col.replace('Geography_', '') for col in encoded_geo_cols_with_prefix}

    # Convert the one-hot encoded array to a DataFrame and rename columns
    one_hot_geo_df = pd.DataFrame(one_hot_geo, index=processed_df.index, columns=encoded_geo_cols_with_prefix).rename(columns=rename_dict)

    combined_processed_df = pd.concat([scaled_numeric_df, one_hot_geo_df], axis=1)

    # Select and reorder columns to match the training data (X_train)
    X_input = combined_processed_df[X_train.columns]

    # Predict probabilities
    prob = model.predict_proba(X_input)[:, 1]

In [868]:
input_df_example = raw_df[:5].copy()
probabilities_train = predict_raw_df(scaler, enc, numeric_cols, categorical_cols, model, input_df_example)
display(probabilities)

array([1.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
       8.96460201e-90, 1.00000000e+00, 0.00000000e+00])

11. Зчитайте дані з `test.csv` в змінну `test_raw_df`. Зробіть передбачення для кожного рядка даних з функцією з попереднього завдання і запишіть результат в нову колонку `Exited` в датафреймі `test_raw_df`.

In [841]:
dataset_url = 'https://www.kaggle.com/competitions/bank-customer-churn-prediction-dlu-course-c-3/data?select=test.csv'

In [844]:
od.download(dataset_url)


Skipping, found downloaded files in "./bank-customer-churn-prediction-dlu-course-c-3" (use force=True to force download)


In [852]:
data_dir = './bank-customer-churn-prediction-dlu-course-c-3'

In [853]:
os.listdir(data_dir)

['sample_submission.csv', 'test.csv', 'train.csv']

In [857]:
test_csv_path = data_dir + '/test.csv'

In [858]:
test_raw_df = pd.read_csv(test_csv_path)

In [860]:
test_raw_df

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,15000,15594796.0,Chu,584.0,Germany,Male,30.0,2.0,146053.66,1.0,1.0,1.0,157891.86
1,15001,15642821.0,Mazzi,551.0,France,Male,39.0,5.0,0.00,2.0,1.0,1.0,67431.28
2,15002,15716284.0,Onyekachi,706.0,France,Male,43.0,8.0,0.00,2.0,1.0,0.0,156768.45
3,15003,15785078.0,Martin,717.0,Spain,Male,45.0,3.0,0.00,1.0,1.0,1.0,166909.87
4,15004,15662955.0,Kenechukwu,592.0,Spain,Male,43.0,8.0,0.00,2.0,1.0,1.0,143681.97
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,24995,15642997.0,Chukwumaobim,639.0,Spain,Male,38.0,10.0,0.00,2.0,1.0,1.0,49637.65
9996,24996,15739271.0,Clements,678.0,Spain,Male,39.0,9.0,0.00,2.0,1.0,1.0,142513.50
9997,24997,15756743.0,Chidiebere,774.0,France,Male,30.0,9.0,0.00,2.0,1.0,0.0,4861.72
9998,24998,15680167.0,Yermakova,595.0,France,Male,38.0,6.0,144875.79,1.0,1.0,0.0,126469.09


In [871]:
test_input_df = test_raw_df.copy()
probabilities_test = predict_raw_df(scaler, enc, numeric_cols, categorical_cols, model, test_input_df)
test_raw_df

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,15000,15594796.0,Chu,584.0,Germany,Male,30.0,2.0,146053.66,1.0,1.0,1.0,157891.86
1,15001,15642821.0,Mazzi,551.0,France,Male,39.0,5.0,0.00,2.0,1.0,1.0,67431.28
2,15002,15716284.0,Onyekachi,706.0,France,Male,43.0,8.0,0.00,2.0,1.0,0.0,156768.45
3,15003,15785078.0,Martin,717.0,Spain,Male,45.0,3.0,0.00,1.0,1.0,1.0,166909.87
4,15004,15662955.0,Kenechukwu,592.0,Spain,Male,43.0,8.0,0.00,2.0,1.0,1.0,143681.97
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,24995,15642997.0,Chukwumaobim,639.0,Spain,Male,38.0,10.0,0.00,2.0,1.0,1.0,49637.65
9996,24996,15739271.0,Clements,678.0,Spain,Male,39.0,9.0,0.00,2.0,1.0,1.0,142513.50
9997,24997,15756743.0,Chidiebere,774.0,France,Male,30.0,9.0,0.00,2.0,1.0,0.0,4861.72
9998,24998,15680167.0,Yermakova,595.0,France,Male,38.0,6.0,144875.79,1.0,1.0,0.0,126469.09


In [872]:
test_raw_df['Exited'] = probabilities_test
test_raw_df

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,15000,15594796.0,Chu,584.0,Germany,Male,30.0,2.0,146053.66,1.0,1.0,1.0,157891.86,1.000000e+00
1,15001,15642821.0,Mazzi,551.0,France,Male,39.0,5.0,0.00,2.0,1.0,1.0,67431.28,0.000000e+00
2,15002,15716284.0,Onyekachi,706.0,France,Male,43.0,8.0,0.00,2.0,1.0,0.0,156768.45,0.000000e+00
3,15003,15785078.0,Martin,717.0,Spain,Male,45.0,3.0,0.00,1.0,1.0,1.0,166909.87,0.000000e+00
4,15004,15662955.0,Kenechukwu,592.0,Spain,Male,43.0,8.0,0.00,2.0,1.0,1.0,143681.97,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,24995,15642997.0,Chukwumaobim,639.0,Spain,Male,38.0,10.0,0.00,2.0,1.0,1.0,49637.65,1.316499e-306
9996,24996,15739271.0,Clements,678.0,Spain,Male,39.0,9.0,0.00,2.0,1.0,1.0,142513.50,0.000000e+00
9997,24997,15756743.0,Chidiebere,774.0,France,Male,30.0,9.0,0.00,2.0,1.0,0.0,4861.72,8.964602e-90
9998,24998,15680167.0,Yermakova,595.0,France,Male,38.0,6.0,144875.79,1.0,1.0,0.0,126469.09,1.000000e+00


12. Зчитайте файл 'sample_submission.csv' і зповніть колонку `Exited` передбаченими вашою моделлю значеннями для кожного відповідного `id` клієнта.
Виведіть перші 5 рядків фінального датасету, збережіть результат в файл `submission_log_reg.csv` (УВАГА! не запишіть випадково ще індекси рядків!) і здайте цей файл на Kaggle зі свого профіля натиснувши `Submit Prediction`.

In [876]:
submission_file_path = data_dir + '/sample_submission.csv'
submission = pd.read_csv(submission_file_path)
submission['Exited']=probabilities_test
submission.head()

,id,Exited
0,15000,1.0
1,15001,0.0
2,15002,0.0
3,15003,0.0
4,15004,0.0


In [878]:
output_filename = 'submission_log_reg.csv'
submission.to_csv(output_filename, index=False)